# Building a structured filled DEM from GeoTIFF files

<div class="alert alert-block alert-info"> GeoTIFF files are raster dataset embedding Geographic Information (latitude, longitude, map projection etc.) The geographic data of a GeoTIFF file can be used to position the image in the correct location and geometry and build structured and unstructured meshes used in our landscape evolution model.  

In this example, we use a dataset of the Blue Mountains region (100 km West of Sydney AU) that can be extracted from the ASTER Global DEM from USGS Global Data Explorer website: 
**https://gdex.cr.usgs.gov/gdex/**</div>


You can find an non-exhaustive list of available digital elvation dataset in the following website: 
+ https://github.com/openterrain/openterrain/wiki/Terrain-Data

***

<img src="images/bluemountains.png" width="70%">

***

## Notebook contents

   - [Converting from lon/lat to metres](#Converting-from-lon/lat-to-metres)
   - [Clipped elevation grid](#Clipped-elevation-grid)
   - [Structured elevation grid](#Structured-elevation-grid)
   - [Perform pit filling](#Perform-pit-filling)
   - [Priority-flood + epsilon](#Priority-flood-+-epsilon)
   


In [ ]:
import pycpt
import lavavu

import meshio
import numpy as np
import pygmsh as pg
import fillit as pitfill

from time import clock

from scipy.interpolate import RectBivariateSpline

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

label_size = 8
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

# Converting from lon/lat to metres

To reproject the grid from lon/lat coordinates to UTM (metres), two main libraries are available within the Docker image:

+ `pygeotools` -- https://github.com/dshean/pygeotools
+ `rasterio` -- https://github.com/mapbox/rasterio

First, we specify our DEM filename:

In [ ]:
filename = 'Data/BlueMountains.tif'

If `pygeotools` is used, you will load the `warplib` library and will need to define the resolution `utmRes` and the **coordinate system** you want to used for your new projection `utmProj`. 

As an example, here we use the UTM zone 55 for the southern hemisphere based on the location of our dataset. 
Once the `warplib` library has done the projection transformation you will be able to access the projected dataset as a numpy masked array using the `iolib` library from `pygeotools`.


It will typically be used as follows:

``` python
from pygeotools.lib import iolib,geolib,warplib

utmRes = 30.
utmProj = "+proj=utm +zone=55 +south +datum=WGS84 +ellps=WGS84 "
utmData = warplib.diskwarp_multi_fn( [filename], t_srs = utmProj, res = utmRes)
utm = iolib.ds_getma(utmData[0]) 

```

Below, we show how this can be done using `rasterio`. First we load the required libraries and then define the requested projection (here we used **EPSG reference** for the region `EPSG:32756` - available from http://spatialreference.org/ref/epsg/). 

In [ ]:
import rasterio
from rasterio import crs
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Name of the reprojected UTM file to be created
outputfile = 'Data/BlueMountainsUTM.tif'

# Reproject to UTM zone 55 S 
dst_crs = {'init': 'EPSG:32756'}

# Requested reprojected dataset resolution (metres)
utmRes = 50.

We then use the following cell to make the projection and get the interpolated elevation points at the requested resolution (`elev` -- a numpy masked array)

In [ ]:
with rasterio.drivers(CHECK_WITH_INVERT_PROJ=True):
    with rasterio.open(filename) as src:
        
        profile = src.profile
        if src.nodata is None:
            nodata = -32768.0
        else:
            nodata = src.nodata
        
        # Calculate the ideal dimensions and transformation in the new crs
        dst_affine, dst_width, dst_height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds, resolution=utmRes)

        # update the relevant parts of the profile
        profile.update({
            'crs': dst_crs,
            'transform': dst_affine,
            'affine': dst_affine,
            'width': dst_width,
            'height': dst_height
        })

        # Reproject and write each band
        with rasterio.open(outputfile, 'w', **profile) as dst:
            src_array = src.read()
            dst_array = np.empty((int(dst_height), int(dst_width)), dtype='int16')

            reproject(
                    # Source parameters
                    source=src_array,
                    src_crs=src.crs,
                    src_transform=src.affine,
                    src_nodata=nodata,
                
                    # Destination paramaters
                    destination=dst_array,
                    dst_transform=dst_affine,
                    dst_crs=dst_crs,
                    dst_nodata=nodata,
                
                    # Configuration
                    resampling=Resampling.nearest,
                    num_threads=2)

            dst.write(dst_array,1)
            elev = np.ma.masked_where(dst_array == nodata, dst_array)

We can look at the `metadata` associated with the new GeoTIFF file using for example:

In [ ]:
dst.meta

# Clipped elevation grid

We can visualise the new elevation array using the following function:

In [ ]:
def plotElevation( data, cmin, cmax, colormap):
    '''
    data: dataset to plot
    cmin,cmax: extent of the colormap
    colormap: color scale to use   
    '''
    
    # Figure size is defined here
    fig = plt.figure(1, figsize=(6,6))
    
    ax = plt.gca()
    im = ax.imshow(np.flipud(data), interpolation='nearest', cmap=colormap,
                     vmin=cmin, vmax=cmax)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.1)
    cbar = plt.colorbar(im,cax=cax)
    plt.tight_layout()

    plt.show()
    plt.close(fig)

We choose to use `pycpt` -- Python tools to load and handle **cpt** (GMT format) color maps for use with matplotlib (_e.g._ from cpt-city)

You can pick a colorbar from the following website:
+ http://soliton.vm.bytemark.co.uk/pub/cpt-city/index.html

In [ ]:
topocmap = pycpt.load.cmap_from_cptcity_url('td/DEM_screen.cpt')
plotElevation( elev, 0, 1300, topocmap)

As you can see from the figure above, we will need to **clip** our array to remove the `nodata` values induced by the reprojection... We do that by just selecting the extent of the rows and columns number from our `elev` numpy array...

In [ ]:
plotElevation( elev[80:-80,100:-100], 0, 1300, topocmap)

This extent seems to work :-), let's see if we still have some masked values

In [ ]:
np.ma.is_masked(elev[80:-80,100:-100])

We then defined a new elevation array `dem` based on the clipped one:

In [ ]:
dem = elev[80:-80,100:-100]

# Structured elevation grid

To keep your coordinates system for post-processing and to potentially reproject the outputs from the landscape evolution model in another geospatial system we needs to specify the X and Y axes.

We do it like this:

In [ ]:
xMin = dst.transform[2]
xMax = dst.transform[2] + abs(dst.transform[0])*dst_width

yMin = dst.transform[5] - abs(dst.transform[4])*dst_height
yMax = dst.transform[5]

print("Initial DEM:\n")

print("Lower left coordinates       Xmin: {}, Ymin: {}".format(xMin,yMin))
print("Upper right coordinates      Xmax: {}, Ymax: {}".format(xMax,yMax))

We can now create the X and Y coordinates, at this point we can choose to decrease the resolution if needed by using the `step` parameter (integer)

In [ ]:
step = 1
spacing = utmRes*step

nZ = dem[::step,::step]

# To applying a smoothing filter on the processed DEM you
# might want to use the following scipy filter
from scipy.ndimage.filters import gaussian_filter
nZ = gaussian_filter(nZ, sigma=2)

# Perform pit filling

We will now perform pit filling using one of the available priority-flood algorithms available in `fillit`.

For regular grids, the following `classes` are available:
+ `depressionFillingZhou`
+ `depressionFillingBarnes`

***

These 2 classes are based on the following papers:

Barnes, Lehman, Mulla. "Priority-Flood: An Optimal Depression-Filling and
Watershed-Labeling Algorithm for Digital Elevation Models". Computers & Geosciences.
Vol 62, Jan 2014, pp 117–127 - [link](https://www.sciencedirect.com/science/article/pii/S0098300413001337)

Zhou, Sun, Fu. "An efficient variant of the Priority-Flood algorithm for filling
depressions in raster digital elevation models". Computers & Geosciences.
Vol 90, Feb 2016, pp 87–96 - [link](https://www.sciencedirect.com/science/article/pii/S0098300416300553)

***

To call one of these classes, you will typically do as follows:

``` python
import fillit as pitfill

# Class initialisation
pitClass = pitfill.depressionFillingZhou(dem)

# Performing pit filling
fillZ = pitClass.performPitFillingStruct()

```

Here we illustrate how this is done for the 2 classes...

### Barnes (2014)

In [ ]:
t0 = clock()
pitClass = pitfill.depressionFillingBarnes(nZ,sealevel=10.)
fillBarnes = pitClass.performPitFillingStruct()
print('Pit filling based on Barnes\'Algorithm (%0.02f seconds)' % (clock() - t0))

### Zhou (2016)

It is worth mentionning that the performance of Zhou's algorithm in `fillit` performed a bit slower than the Barnes'one mainly because it also computes additional information such as pit labelling, pit volume evaluation, watershed definition and spill-over graph...

In [ ]:
t0 = clock()
pitClass = pitfill.depressionFillingZhou(nZ,sealevel=10.)
fillZhou = pitClass.performPitFillingStruct()
print('Pit filling based on Zhou\'s Algorithm (%0.02f seconds)' % (clock() - t0))

Comparisons between the 2 algorithms:

In [ ]:
print('+ Pit filling elevation difference between Zhou & Barnes: \n \t \t min: %0.02f - max: %0.02f \n' % ((fillBarnes-fillZhou).min(),(fillBarnes-fillZhou).max()))

We can visualise the filled elevation differences with the initial digital elevation model

In [ ]:
topocmap = pycpt.load.cmap_from_cptcity_url('cmocean/amp.cpt')
plotElevation(fillZhou-nZ, 0, 20, topocmap)

We can now create an elevation grid with this filled array

In [ ]:
nx = fillBarnes.shape[1]
ny = fillBarnes.shape[0]

minX, maxX = xMin, xMin+spacing*nx
minY, maxY = yMin, yMin+spacing*ny

xcoords = np.arange(minX, maxX, spacing)
ycoords = np.arange(minY, maxY, spacing)

X, Y = np.meshgrid(xcoords, ycoords)

coords = np.vstack([X.ravel(), Y.ravel()])
coords = np.vstack([coords, fillBarnes.ravel()]).T


print("Clipped DEM:\n")

print("Resolution (m)            res: {}".format(spacing))
print("Number of points         nbpt: {}".format(coords.shape[0]))
print("Elevation map shape        nx: {}, ny: {}\n".format(nx,ny))

print("Lower left coordinates   Xmin: {}, Ymin: {}".format(minX,minY))
print("Upper right coordinates  Xmax: {}, Ymax: {}".format(maxX,maxY))

From there we can directly create our initial surface for running a landscape evolution model using a structured grid. 

Here we use [**LavaVu**](https://github.com/OKaluza/LavaVu) to render in 3D our digital elevation grid within the Jupyter notebook.

By clicking on the wireframe you can see the structured grid that will be used in our simulation.

In [ ]:
lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[700,700], near=-10.0)

lvQuad = lv.quads("QuadMesh", dims=(nx, ny), wireframe=True, colour="#161616", opacity=1.0)
lvQuad.vertices(coords)
lvQuad.values(fillZhou.ravel(), "elevation")
lvQuad.colourmap("geo", range=[-1800.,1800.])

lv.translation(823.959, 14666.457, -48052.191)
lv.rotation(-28.235, 4.249, -0.51)
lv.scale('z', 4)

lv.control.Checkbox(property='axis')
lvQuad.control.Checkbox(property='wireframe', label="wireframe")
lv.control.Panel()
lv.control.ObjectList()
lvQuad.control.Range(command='scale z', range=(1,20), step=1., value=4)
lv.control.show()

We use `meshio` to create a VTK file that will then be passed to the landscape evolution model.

In [ ]:
ptsXYZ = coords.copy() 
ptsXYZ[:,2] = 0.

mesh = meshio.Mesh(ptsXYZ, {'quad':pitClass.getCellConnectivity()}, {'Z':fillZhou.ravel()})
meshio.write("StructGridBlueMountainHRfill.vtk", mesh)

# Priority-flood + epsilon

In cases where one wants to remove flat surfaces the `eps` or $\epsilon$ (for epsilon) parameter can be used. $\epsilon$ is large enough to direct flow, but sufficiently small as to have no other effects on the DEM’s hydrologic properties.

Only the methods from **Barnes** can be used to fill the flat areas.

### Barnes' method

In [ ]:
t0 = clock()
pitClass = pitfill.depressionFillingBarnes(nZ,eps=1.e-6,sealevel=10.)
fillBeps = pitClass.performPitFillingStruct()
print('Pit filling based on Barnes\'Algorithm (%0.02f seconds)' % (clock() - t0))

In [ ]:
topocmap = pycpt.load.cmap_from_cptcity_url('cmocean/amp.cpt')
plotElevation(fillBeps-fillBarnes, 0, (fillBeps-fillBarnes).max()/5., topocmap)

Similarly to what we did previously, we use `meshio` to create a VTK file that will then be passed to the landscape evolution model.

In [ ]:
ptsXYZ = coords.copy() 
ptsXYZ[:,2] = 0.

mesh = meshio.Mesh(ptsXYZ, {'quad':pitClass.getCellConnectivity()}, {'Z':fillBeps.ravel()})
meshio.write("StructGridBlueMountainHReps.vtk", mesh)

We can again use **LavaVu** to render in 3D our digital elevation grid within the Jupyter notebook.

In [ ]:
lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[700,700], near=-10.0)

lvQuad = lv.quads("QuadMesh", dims=(nx, ny), wireframe=True, colour="#161616", opacity=1.0)
lvQuad.vertices(coords)
lvQuad.values(fillBeps.ravel(), "elevation")
lvQuad.colourmap("geo", range=[-1800.,1800.])

lv.translation(823.959, 14666.457, -48052.191)
lv.rotation(-28.235, 4.249, -0.51)
lv.scale('z', 4)

lv.control.Checkbox(property='axis')
lvQuad.control.Checkbox(property='wireframe', label="wireframe")
lv.control.Panel()
lv.control.ObjectList()
lvQuad.control.Range(command='scale z', range=(1,20), step=1., value=4)
lv.control.show()